<a href="https://colab.research.google.com/github/srinihash/understandings_and_projects/blob/master/aws_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://realpython.com/python-boto3-aws-s3/

In [1]:
!pip install boto3

In [3]:
# https://stackoverflow.com/questions/54571009/how-to-hide-secret-keys-in-google-colaboratory-from-users-having-the-sharing-lin
# Mounting my drive (do not worry, no one ca hack this because "login" is required for running this Colab)
!mkdir -p ~/.aws
!cp "/content/drive/My Drive/aws/config" ~/.aws/config

In [8]:
# Client connection
import boto3
s3_client = boto3.client('s3')
print(dir(s3_client))

['_PY_TO_OP_NAME', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cache', '_client_config', '_convert_to_request_dict', '_emit_api_params', '_endpoint', '_exceptions', '_exceptions_factory', '_get_waiter_config', '_load_exceptions', '_loader', '_make_api_call', '_make_request', '_register_handlers', '_request_signer', '_response_parser', '_serializer', '_service_model', 'abort_multipart_upload', 'can_paginate', 'complete_multipart_upload', 'copy', 'copy_object', 'create_bucket', 'create_multipart_upload', 'delete_bucket', 'delete_bucket_analytics_configuration', 'delete_bucket_cors', 'delete_bucket_encryption', 'delete_bucket_inventory_configuration', 'delete_bucket_lifecycle',

In [9]:
# Direct Resource connection
import boto3
s3_resource = boto3.resource('s3')
print(dir(s3_resource))

['Bucket', 'BucketAcl', 'BucketCors', 'BucketLifecycle', 'BucketLifecycleConfiguration', 'BucketLogging', 'BucketNotification', 'BucketPolicy', 'BucketRequestPayment', 'BucketTagging', 'BucketVersioning', 'BucketWebsite', 'MultipartUpload', 'MultipartUploadPart', 'Object', 'ObjectAcl', 'ObjectSummary', 'ObjectVersion', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'buckets', 'create_bucket', 'get_available_subresources', 'meta']


boto3 is dependent upon lower library botocore : https://github.com/boto/botocore/tree/master

In [12]:
import uuid
def create_bucket_name(bucket_prefix):
    # The generated bucket name must be between 3 and 63 chars long
    return ''.join([bucket_prefix, str(uuid.uuid4())])

def create_bucket(bucket_prefix, s3_connection):
    session = boto3.session.Session()
    current_region = session.region_name
    bucket_name = create_bucket_name(bucket_prefix)
    bucket_response = s3_connection.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={
        'LocationConstraint': current_region})
    print(bucket_name, current_region)
    return bucket_name, bucket_response

def create_temp_file(size, file_name, file_content):
    random_file_name = ''.join([str(uuid.uuid4().hex[:6]), file_name])
    with open(random_file_name, 'w') as f:
        f.write(str(file_content) * size)
    return random_file_name


first_bucket_name, first_response = create_bucket(bucket_prefix='firstpythonbucket',
                                                  s3_connection=s3_resource.meta.client)

second_bucket_name, second_response = create_bucket(bucket_prefix='secondpythonbucket',
                                                    s3_connection=s3_resource)

print("first bucket : {} :: second bucket : {}".format(first_bucket_name, 
                                                       second_bucket_name))

first_file_name = create_temp_file(300, 'firstfile.txt', 'f')

first_bucket = s3_resource.Bucket(name=first_bucket_name)
first_object = s3_resource.Object(bucket_name=first_bucket_name, 
                                  key=first_file_name)

firstpythonbucket06258cc2-e5b9-419b-b547-de43392987d2 eu-west-1
secondpythonbucket7e3fe8d5-12c7-4d3d-8992-dd59a9294707 eu-west-1
first bucket : firstpythonbucket06258cc2-e5b9-419b-b547-de43392987d2 :: second bucket : secondpythonbucket7e3fe8d5-12c7-4d3d-8992-dd59a9294707


Let's now upload an object to the created bucket, in this case a file !
We can upload the same using 3 constructs:
1. From a Object instance
2. From a Bucket instance
3. From the client

In [14]:
# Object instance version
s3_resource.Object(first_bucket_name, first_file_name).upload_file(Filename=first_file_name)

first_object.upload_file(first_file_name)

In [15]:
# Bucket instance version
s3_resource.Bucket(first_bucket_name).upload_file(Filename=first_file_name, Key=first_file_name) 

In [16]:
# Client version
s3_resource.meta.client.upload_file(Filename=first_file_name, Bucket=first_bucket_name, Key=first_file_name) 

Let's copy this Object from one bucket to another

In [21]:
def copy_to_bucket(bucket_from_name, bucket_to_name, file_name):
    copy_resource = {
        'Bucket': bucket_from_name,
        'Key': file_name 
    }
    s3_resource.Object(bucket_to_name, file_name).copy(copy_resource)

copy_to_bucket(first_bucket_name, second_bucket_name, first_file_name)

In [22]:
s3_resource.Object(second_bucket_name, first_file_name).delete()

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Sat, 08 Aug 2020 10:16:43 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'vbVZNEZMl0pwckgtQdyBmoSbpx1EC5h6yNqmr9LDbd899+EyvuZRgf3Tap8BKtNW6KotVIvdqlQ=',
   'x-amz-request-id': 'F98C49F4023AD6EF'},
  'HTTPStatusCode': 204,
  'HostId': 'vbVZNEZMl0pwckgtQdyBmoSbpx1EC5h6yNqmr9LDbd899+EyvuZRgf3Tap8BKtNW6KotVIvdqlQ=',
  'RequestId': 'F98C49F4023AD6EF',
  'RetryAttempts': 0}}

In [23]:
second_file_name = create_temp_file(400, 'secondfile.txt', 's')
second_object = s3_resource.Object(first_bucket_name, second_file_name)
second_object.upload_file(second_file_name, ExtraArgs={'ACL': 'public-read'})
second_object_acl = second_object.Acl()

In [24]:
second_object_acl.grants

[{'Grantee': {'DisplayName': 'sriniaws24',
   'ID': '00bba0ab9eddfd9339729ddc4c4e63f5ceb9cdf71132f46d8a3c7998e4fde7a0',
   'Type': 'CanonicalUser'},
  'Permission': 'FULL_CONTROL'},
 {'Grantee': {'Type': 'Group',
   'URI': 'http://acs.amazonaws.com/groups/global/AllUsers'},
  'Permission': 'READ'}]

Encrypting your objects

In [26]:
third_file_name = create_temp_file(300, 'thirdfile.txt', 't')
third_object = s3_resource.Object(first_bucket_name, third_file_name)
third_object.upload_file(third_file_name, ExtraArgs={'ServerSideEncryption': 'AES256',
                                                     'ACL': 'public-read'})

In [27]:
third_object.server_side_encryption

'AES256'

In [31]:
third_object.storage_class

Versioning your objects

In [32]:
def enable_bucket_versioning(bucket_name):
    bkt_versioning = s3_resource.BucketVersioning(bucket_name)
    bkt_versioning.enable()
    print(bkt_versioning.status)

enable_bucket_versioning(first_bucket_name)

Enabled


In [33]:
first_bucket_name

'firstpythonbucket06258cc2-e5b9-419b-b547-de43392987d2'

In [36]:
s3_resource.Object(first_bucket_name, first_file_name).upload_file(first_file_name)
s3_resource.Object(first_bucket_name, first_file_name).upload_file(third_file_name)

In [37]:
s3_resource.Object(first_bucket_name, first_file_name).version_id

'LRil_k4aX4uHvixY3ctPEir3fXeXC8RK'

Let's traverse contents of S3 of the account here 

In [38]:
for bucket in s3_resource.buckets.all():
    print(bucket.name)

elasticbeanstalk-eu-west-1-495542595299
firstpythonbucket06258cc2-e5b9-419b-b547-de43392987d2
secondpythonbucket7e3fe8d5-12c7-4d3d-8992-dd59a9294707


Traversing all contents of bucket

In [39]:
for obj in first_bucket.objects.all():
    print(obj.key)

0e83c3thirdfile.txt
860b06thirdfile.txt
d52757secondfile.txt
e2d9e8firstfile.txt


In [42]:
for obj in first_bucket.objects.all():
    sub_rsc = obj.Object()
    print(obj.key, obj.storage_class, obj.last_modified, sub_rsc.version_id, sub_rsc.metadata)

0e83c3thirdfile.txt STANDARD 2020-08-08 11:14:35+00:00 null {}
860b06thirdfile.txt STANDARD 2020-08-08 11:13:32+00:00 null {}
d52757secondfile.txt STANDARD 2020-08-08 11:06:40+00:00 null {}
e2d9e8firstfile.txt STANDARD 2020-08-08 11:47:38+00:00 LRil_k4aX4uHvixY3ctPEir3fXeXC8RK {}


Deleting Buckets and Objects

In [46]:
def delete_all_objects(bucket_name):
    res = []
    bucket = s3_resource.Bucket(first_bucket_name)
    for obj_version in bucket.object_versions.all():
        res.append({'Key': obj_version.object_key,
                    'VersionId': obj_version.id})
    print(res)
    bucket.delete_objects(Delete={'Objects': res})

delete_all_objects(first_bucket_name)

[{'Key': '0e83c3thirdfile.txt', 'VersionId': 'null'}, {'Key': '860b06thirdfile.txt', 'VersionId': 'null'}, {'Key': 'd52757secondfile.txt', 'VersionId': 'null'}, {'Key': 'e2d9e8firstfile.txt', 'VersionId': 'LRil_k4aX4uHvixY3ctPEir3fXeXC8RK'}, {'Key': 'e2d9e8firstfile.txt', 'VersionId': 'Y7fS5dG4y6YhQvG7mCbT4vH4oRvvSkDz'}, {'Key': 'e2d9e8firstfile.txt', 'VersionId': 'xxXLl8X8.6jHLe5dZtzVgS_IIhYxz39f'}, {'Key': 'e2d9e8firstfile.txt', 'VersionId': 'IskhhOxK8j.qIqK1Q54sxu5x9ud3zevw'}, {'Key': 'e2d9e8firstfile.txt', 'VersionId': 'null'}]


In [47]:
s3_resource.Bucket(first_bucket_name).delete()

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Sat, 08 Aug 2020 12:26:06 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'njn40F3XRWmizDRMH/UAfhoEBn5wuVuPCmlzigY3g+AeTdE/qyAJhoDZuItRmQ8TtcMN+bDOQJA=',
   'x-amz-request-id': '39FF2CC6B2B6EA8B'},
  'HTTPStatusCode': 204,
  'HostId': 'njn40F3XRWmizDRMH/UAfhoEBn5wuVuPCmlzigY3g+AeTdE/qyAJhoDZuItRmQ8TtcMN+bDOQJA=',
  'RequestId': '39FF2CC6B2B6EA8B',
  'RetryAttempts': 0}}

In [48]:
s3_resource.Bucket(second_bucket_name).delete()

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Sat, 08 Aug 2020 12:26:41 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'jSAlBuQ+i5GHQFW/xOAzMVGw4Gona2yE98GJHCNB+VQub0MOWzHuCkAb/D8EY0xopatkCZOsv7o=',
   'x-amz-request-id': '5C179278EFEF2282'},
  'HTTPStatusCode': 204,
  'HostId': 'jSAlBuQ+i5GHQFW/xOAzMVGw4Gona2yE98GJHCNB+VQub0MOWzHuCkAb/D8EY0xopatkCZOsv7o=',
  'RequestId': '5C179278EFEF2282',
  'RetryAttempts': 0}}